**Content based Movie Recommendation System for Netflix OTT Platform**

In [110]:
#import the required libraries for the uploading data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk 
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [111]:
# read the movie dataset
dataset=pd.read_csv("/content/drive/MyDrive/IMDBmoviedataset.csv")
dataset.head(5)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,tomatoURL
0,The Player,2015,TV-14,24 Sep 2015,60 min,"Action, Crime, Drama",NaN,"John Fox, John Rogers","Philip Winchester, Damon Gupton, Wesley Snipes...","A former intelligence and FBI officer, who now...",English,USA,1 nomination.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,7.1/10,NaN,7.1,"6,331",tt4474310,series,NaN,NaN,NaN,NaN,True,NaN
1,The Honeymooners,1955–1956,NaN,01 Oct 1955,30 min,"Comedy, Family",NaN,Jackie Gleason,"Jackie Gleason, Art Carney, Audrey Meadows, Jo...",A bus driver and his sewer worker friend strug...,English,USA,Won 1 Primetime Emmy. Another 3 wins & 7 nomin...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.7/10,NaN,8.7,"3,637",tt0042114,series,23 Sep 2003,NaN,NaN,NaN,True,http://www.rottentomatoes.com/m/ralphs-world-s...
2,McHale's Navy,1962–1966,TV-G,11 Oct 1962,30 min,"Comedy, War",NaN,NaN,"Ernest Borgnine, Joe Flynn, Tim Conway, Carl B...",The misadventures of a misfit PT Boat crew dur...,English,USA,Nominated for 5 Primetime Emmys. Another 1 nom...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,7.5/10,NaN,7.5,"1,698",tt0055689,series,NaN,NaN,NaN,NaN,True,NaN
3,Bewitched,1964–1972,TV-G,17 Sep 1964,25 min,"Comedy, Family, Fantasy",NaN,Sol Saks,"Elizabeth Montgomery, Agnes Moorehead, David W...",A witch married to an ordinary man cannot resi...,English,USA,Nominated for 4 Golden Globes. Another 13 wins...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,7.6/10,NaN,7.6,"12,399",tt0057733,series,NaN,NaN,NaN,NaN,True,NaN
4,M*A*S*H,1972–1983,TV-PG,17 Sep 1972,25 min,"Comedy, Drama, War",NaN,Larry Gelbart,"Alan Alda, Loretta Swit, Jamie Farr, William C...",The staff of an army hospital in the Korean wa...,English,USA,Won 8 Golden Globes. Another 54 wins & 153 nom...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.4/10,NaN,8.4,"40,349",tt0068098,series,NaN,NaN,NaN,NaN,True,NaN


In [112]:
#load the movie dataset via google colab
#from google.colab import files
#uploaded = files.upload()
#dataset= pd.read_csv("Moviedataset.csv")

In [113]:
# print the columns present in the  movie datatset
print(dataset.columns)

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response', 'tomatoURL'],
      dtype='object')


In [114]:
# seperate required columns ['Director','Genre','Actors','Plot','Title'] from the original dataset 
moviedata = dataset.loc[:,['Genre','Actors','Plot','Title']]
moviedata.head()

,Genre,Actors,Plot,Title
0,"Action, Crime, Drama","Philip Winchester, Damon Gupton, Wesley Snipes...","A former intelligence and FBI officer, who now...",The Player
1,"Comedy, Family","Jackie Gleason, Art Carney, Audrey Meadows, Jo...",A bus driver and his sewer worker friend strug...,The Honeymooners
2,"Comedy, War","Ernest Borgnine, Joe Flynn, Tim Conway, Carl B...",The misadventures of a misfit PT Boat crew dur...,McHale's Navy
3,"Comedy, Family, Fantasy","Elizabeth Montgomery, Agnes Moorehead, David W...",A witch married to an ordinary man cannot resi...,Bewitched
4,"Comedy, Drama, War","Alan Alda, Loretta Swit, Jamie Farr, William C...",The staff of an army hospital in the Korean wa...,M*A*S*H


In [115]:
# checking for the any missing value in the data
print(moviedata.isnull())
moviedata.info()

      Genre  Actors   Plot  Title
0     False   False  False  False
1     False   False  False  False
2     False   False  False  False
3     False   False  False  False
4     False   False  False  False
...     ...     ...    ...    ...
5377  False    True  False  False
5378  False   False  False  False
5379  False   False  False  False
5380  False   False  False  False
5381  False   False  False  False

[5382 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5382 entries, 0 to 5381
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Genre   5379 non-null   object
 1   Actors  5375 non-null   object
 2   Plot    5368 non-null   object
 3   Title   5382 non-null   object
dtypes: object(4)
memory usage: 168.3+ KB


In [116]:
# Replace the NaN value by the "is" i.e. by a stopword so that it can be removed during tokenization
df=moviedata.replace(np.nan, "is")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5382 entries, 0 to 5381
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Genre   5382 non-null   object
 1   Actors  5382 non-null   object
 2   Plot    5382 non-null   object
 3   Title   5382 non-null   object
dtypes: object(4)
memory usage: 168.3+ KB


In [117]:
# concatenate the columns to form a single for describing the moview
df["Content"] =  df["Genre"]+ "" +df['Actors']+""+df['Plot']
print(df.head(10))

                      Genre  ...                                            Content
0      Action, Crime, Drama  ...  Action, Crime, DramaPhilip Winchester, Damon G...
1            Comedy, Family  ...  Comedy, FamilyJackie Gleason, Art Carney, Audr...
2               Comedy, War  ...  Comedy, WarErnest Borgnine, Joe Flynn, Tim Con...
3   Comedy, Family, Fantasy  ...  Comedy, Family, FantasyElizabeth Montgomery, A...
4        Comedy, Drama, War  ...  Comedy, Drama, WarAlan Alda, Loretta Swit, Jam...
5      Action, Crime, Drama  ...  Action, Crime, DramaKarl Malden, Michael Dougl...
6  Action, Adventure, Crime  ...  Action, Adventure, CrimeGeorge Peppard, Dwight...
7      Action, Crime, Drama  ...  Action, Crime, DramaDon Johnson, Philip Michae...
8             Drama, Family  ...  Drama, FamilyMegan Follows, Colleen Dewhurst, ...
9            Drama, Romance  ...  Drama, RomanceKatherine Kelly Lang, Ronn Moss,...

[10 rows x 5 columns]


**Data Preprocessing steps:**

In [118]:
# import regular expression library for removing symbols/ commas or irrelavent words and tokenized the content
import re

def tokenization(text):
  tokens= re.split('\W+', text)
  return tokens
  
df['tokenize_content'] = df['Content'].apply(lambda x: tokenization(x.lower()))
df.head(5)

,Genre,Actors,Plot,Title,Content,tokenize_content
0,"Action, Crime, Drama","Philip Winchester, Damon Gupton, Wesley Snipes...","A former intelligence and FBI officer, who now...",The Player,"Action, Crime, DramaPhilip Winchester, Damon G...","[action, crime, dramaphilip, winchester, damon..."
1,"Comedy, Family","Jackie Gleason, Art Carney, Audrey Meadows, Jo...",A bus driver and his sewer worker friend strug...,The Honeymooners,"Comedy, FamilyJackie Gleason, Art Carney, Audr...","[comedy, familyjackie, gleason, art, carney, a..."
2,"Comedy, War","Ernest Borgnine, Joe Flynn, Tim Conway, Carl B...",The misadventures of a misfit PT Boat crew dur...,McHale's Navy,"Comedy, WarErnest Borgnine, Joe Flynn, Tim Con...","[comedy, warernest, borgnine, joe, flynn, tim,..."
3,"Comedy, Family, Fantasy","Elizabeth Montgomery, Agnes Moorehead, David W...",A witch married to an ordinary man cannot resi...,Bewitched,"Comedy, Family, FantasyElizabeth Montgomery, A...","[comedy, family, fantasyelizabeth, montgomery,..."
4,"Comedy, Drama, War","Alan Alda, Loretta Swit, Jamie Farr, William C...",The staff of an army hospital in the Korean wa...,M*A*S*H,"Comedy, Drama, WarAlan Alda, Loretta Swit, Jam...","[comedy, drama, waralan, alda, loretta, swit, ..."


In [119]:
# removed the stopwords from the data
stop_words = nltk.corpus.stopwords.words("english")

def remove_stopwords(tokenized_text):
  clean_text = [word for word in tokenized_text if word not in stop_words]
  return clean_text
  
df["clean_content"] = df['tokenize_content'].apply(lambda x:remove_stopwords(x))
df.head(5)


,Genre,Actors,Plot,Title,Content,tokenize_content,clean_content
0,"Action, Crime, Drama","Philip Winchester, Damon Gupton, Wesley Snipes...","A former intelligence and FBI officer, who now...",The Player,"Action, Crime, DramaPhilip Winchester, Damon G...","[action, crime, dramaphilip, winchester, damon...","[action, crime, dramaphilip, winchester, damon..."
1,"Comedy, Family","Jackie Gleason, Art Carney, Audrey Meadows, Jo...",A bus driver and his sewer worker friend strug...,The Honeymooners,"Comedy, FamilyJackie Gleason, Art Carney, Audr...","[comedy, familyjackie, gleason, art, carney, a...","[comedy, familyjackie, gleason, art, carney, a..."
2,"Comedy, War","Ernest Borgnine, Joe Flynn, Tim Conway, Carl B...",The misadventures of a misfit PT Boat crew dur...,McHale's Navy,"Comedy, WarErnest Borgnine, Joe Flynn, Tim Con...","[comedy, warernest, borgnine, joe, flynn, tim,...","[comedy, warernest, borgnine, joe, flynn, tim,..."
3,"Comedy, Family, Fantasy","Elizabeth Montgomery, Agnes Moorehead, David W...",A witch married to an ordinary man cannot resi...,Bewitched,"Comedy, Family, FantasyElizabeth Montgomery, A...","[comedy, family, fantasyelizabeth, montgomery,...","[comedy, family, fantasyelizabeth, montgomery,..."
4,"Comedy, Drama, War","Alan Alda, Loretta Swit, Jamie Farr, William C...",The staff of an army hospital in the Korean wa...,M*A*S*H,"Comedy, Drama, WarAlan Alda, Loretta Swit, Jam...","[comedy, drama, waralan, alda, loretta, swit, ...","[comedy, drama, waralan, alda, loretta, swit, ..."


In [120]:
# import wordnetlemmatizer for words lemmatization 
from nltk.stem import WordNetLemmatizer
lemmtz= nltk.WordNetLemmatizer()

def lemmatization(token_text):
  text = [lemmtz.lemmatize(word) for word in token_text]
  return text
  
df['Final_content'] = df['clean_content'].apply(lambda x: lemmatization(x))
df.head(3)

,Genre,Actors,Plot,Title,Content,tokenize_content,clean_content,Final_content
0,"Action, Crime, Drama","Philip Winchester, Damon Gupton, Wesley Snipes...","A former intelligence and FBI officer, who now...",The Player,"Action, Crime, DramaPhilip Winchester, Damon G...","[action, crime, dramaphilip, winchester, damon...","[action, crime, dramaphilip, winchester, damon...","[action, crime, dramaphilip, winchester, damon..."
1,"Comedy, Family","Jackie Gleason, Art Carney, Audrey Meadows, Jo...",A bus driver and his sewer worker friend strug...,The Honeymooners,"Comedy, FamilyJackie Gleason, Art Carney, Audr...","[comedy, familyjackie, gleason, art, carney, a...","[comedy, familyjackie, gleason, art, carney, a...","[comedy, familyjackie, gleason, art, carney, a..."
2,"Comedy, War","Ernest Borgnine, Joe Flynn, Tim Conway, Carl B...",The misadventures of a misfit PT Boat crew dur...,McHale's Navy,"Comedy, WarErnest Borgnine, Joe Flynn, Tim Con...","[comedy, warernest, borgnine, joe, flynn, tim,...","[comedy, warernest, borgnine, joe, flynn, tim,...","[comedy, warernest, borgnine, joe, flynn, tim,..."


In [121]:
# show the final content after data preprocessing
print(df['Final_content'])

0       [action, crime, dramaphilip, winchester, damon...
1       [comedy, familyjackie, gleason, art, carney, a...
2       [comedy, warernest, borgnine, joe, flynn, tim,...
3       [comedy, family, fantasyelizabeth, montgomery,...
4       [comedy, drama, waralan, alda, loretta, swit, ...
                              ...                        
5377    [documentary, warismoving, political, sphere, ...
5378    [comedy, dramaeric, mabius, kristin, booth, cr...
5379    [comedy, drama, romancele, geng, daniel, henne...
5380    [drama, horror, thrillereva, boehnke, david, c...
5381    [documentarydrew, barrymore, john, august, ste...
Name: Final_content, Length: 5382, dtype: object


In [122]:
# create a copy of data which containing only movie 'title' and  "final content" of the movie data
movie_data = df[['Title', 'Final_content']].copy()
print(movie_data.head(15))


                           Title                                      Final_content
0                     The Player  [action, crime, dramaphilip, winchester, damon...
1               The Honeymooners  [comedy, familyjackie, gleason, art, carney, a...
2                  McHale's Navy  [comedy, warernest, borgnine, joe, flynn, tim,...
3                      Bewitched  [comedy, family, fantasyelizabeth, montgomery,...
4                        M*A*S*H  [comedy, drama, waralan, alda, loretta, swit, ...
5   The Streets of San Francisco  [action, crime, dramakarl, malden, michael, do...
6                     The A-Team  [action, adventure, crimegeorge, peppard, dwig...
7                     Miami Vice  [action, crime, dramadon, johnson, philip, mic...
8           Anne of Green Gables  [drama, familymegan, follows, colleen, dewhurs...
9     The Bold and the Beautiful  [drama, romancekatherine, kelly, lang, ronn, m...
10                       Dekalog  [dramaartur, barcis, olgierd, lukaszewicz,

In [123]:
# import TF-IDF vectorizer for vectorization
from sklearn.feature_extraction.text import TfidfVectorizer  

# vectorize the movie "content" with most significant 30 words  
tfidf = TfidfVectorizer(ngram_range=(1,30), analyzer= remove_stopwords)
tfidf_content = tfidf.fit_transform( movie_data['Final_content'])

# print the shape of vector metrix
print(tfidf_content.shape)
print(tfidf_content)


(5382, 27888)
  (0, 0)	0.0264562540680104
  (0, 24188)	0.16658283064210144
  (0, 4205)	0.15591734143785546
  (0, 2582)	0.18920985112858038
  (0, 19422)	0.14991314985688856
  (0, 20990)	0.15591734143785546
  (0, 19812)	0.16195614398822689
  (0, 25814)	0.11412733776095758
  (0, 25426)	0.23405211254673802
  (0, 2199)	0.14991314985688856
  (0, 13642)	0.14593512007656062
  (0, 17200)	0.17097707690613484
  (0, 3093)	0.14938173111706923
  (0, 19399)	0.20109901831303606
  (0, 17520)	0.12267676529401483
  (0, 20555)	0.17220075488634454
  (0, 26386)	0.15876382759820165
  (0, 14571)	0.1521531692009661
  (0, 8798)	0.17625558569335495
  (0, 22443)	0.16559009648910897
  (0, 27469)	0.1230607769746937
  (0, 18375)	0.13573520644145343
  (0, 9390)	0.1527443026836893
  (0, 13116)	0.17097707690613484
  (0, 10041)	0.12345047792390396
  :	:
  (5381, 20132)	0.23630363900368506
  (5381, 2346)	0.23630363900368506
  (5381, 7066)	0.23630363900368506
  (5381, 3579)	0.22553555029346434
  (5381, 22216)	0.2071273810

In [124]:
# import cosine_similarity library from sk_Learn for similarity matching
from sklearn.metrics.pairwise import cosine_similarity

# Finding cosine similarity between movie contents
cos_similarity = cosine_similarity(tfidf_content, tfidf_content)
print(cos_similarity)

# print the shape of similarity metrix
cos_similarity.shape

[[1.00000000e+00 3.04164705e-02 9.42165746e-04 ... 6.97922179e-04
  1.08553711e-03 7.06666574e-04]
 [3.04164705e-02 1.00000000e+00 6.91421738e-03 ... 5.12180122e-03
  1.32019226e-03 8.59423170e-04]
 [9.42165746e-04 6.91421738e-03 1.00000000e+00 ... 5.66892637e-03
  1.46121889e-03 9.51229160e-04]
 ...
 [6.97922179e-04 5.12180122e-03 5.66892637e-03 ... 1.00000000e+00
  5.71430270e-03 7.04636027e-04]
 [1.08553711e-03 1.32019226e-03 1.46121889e-03 ... 5.71430270e-03
  1.00000000e+00 1.86946295e-02]
 [7.06666574e-04 8.59423170e-04 9.51229160e-04 ... 7.04636027e-04
  1.86946295e-02 1.00000000e+00]]


(5382, 5382)

In [125]:
# assigning the indices to the movie 'Title' column
indices = pd.Series(movie_data.index, index=movie_data["Title"]).drop_duplicates()
indices

Title
The Player                          0
The Honeymooners                    1
McHale's Navy                       2
Bewitched                           3
M*A*S*H                             4
                                 ... 
Return to the Land of Wonders    5377
Signed Sealed Delivered          5378
Shanghai Calling                 5379
A Plague So Pleasant             5380
My Date with Drew                5381
Length: 5382, dtype: int64

In [126]:
#define the function which can suggest top 10 similar movies from the dataset based on sililarity

def recommend(Movie_Title,sig=cos_similarity):
    indx = indices[Movie_Title]
    movie_score = list(enumerate(sig[indx]))
    movie_score = sorted(movie_score,key = lambda x:x[1],reverse=True)
    top_10_similar_movies_indices = movie_score[1:11]
    movies_indices = [i[0] for i in top_10_similar_movies_indices]
    return movie_data['Title'].iloc[movies_indices]
    

**Enter the movie name to watch similar movies from the list:**


In [128]:
# enter the movie name which you are curently watching so that this system can suggest you top 10 similar content movies
Movie_name = str(input("Please Enter the movie name which you are currently watching:"))
print("Following top 10 movies have similar content:-")
recommend(Movie_name)

Please Enter the movie name which you are currently watching:A Plague So Pleasant
Following top 10 movies have similar content:-


4328                     After.Life
4578              Diary of the Dead
45              Friday Night Lights
3041    Metallica Through the Never
1645                        Knowing
3143                   Philadelphia
5058                     Hav Plenty
2407               Dawn of the Dead
5228                   Jesus People
1461                     ParaNorman
Name: Title, dtype: object